In [ ]:
# [Cell 1] 환경 설정 및 라이브러리 import

import os
import json
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# ============================================================
# 경로 및 상수 설정 (recommend_model 폴더 기준)
# ============================================================

# 입력 멜 npy 루트 (Drive for Desktop 경로 - 외부 경로 유지)
MEL_ROOT = Path(r"G:\내 드라이브\Colab Notebooks\mel_data_backup")

# 샤드 저장 루트 (recommend_model 폴더 기준)
SHARD_ROOT = Path("../mel_shards")  # recommend_model 폴더 기준
SHARD_ROOT.mkdir(parents=True, exist_ok=True)

# 고정 길이 T
FIXED_T = 1536

# 샤드당 곡 수
SHARD_SIZE = 500

# 원본 멜 스펙 n_mels (Melon 데이터셋 기준)
ORIGINAL_N_MELS = 48

print(f"📁 MEL_ROOT: {MEL_ROOT}")
print(f"   exists: {MEL_ROOT.exists()}")
print(f"📁 SHARD_ROOT: {SHARD_ROOT}")
print(f"   exists: {SHARD_ROOT.exists()}")
print(f"\n⚙️ FIXED_T: {FIXED_T}")
print(f"⚙️ SHARD_SIZE: {SHARD_SIZE}")

📁 MEL_ROOT: G:\내 드라이브\Colab Notebooks\mel_data_backup
   exists: True
📁 SHARD_ROOT: C:\Users\ASUS\music_recommend\work\mel_shards
   exists: True

⚙️ FIXED_T: 1536
⚙️ SHARD_SIZE: 500


In [2]:
# [Cell 2] 입력 npy 파일 스캔 및 기본 통계 확인

print("🔍 npy 파일 스캔 중...")

npy_paths = []
skipped_count = 0

for p in MEL_ROOT.rglob("*.npy"):
    try:
        sid = int(p.stem)
    except ValueError:
        print(f"[WARN] 숫자가 아닌 파일명, 건너뜀: {p}")
        skipped_count += 1
        continue
    npy_paths.append((sid, p))

# song_id 기준 정렬
npy_paths.sort(key=lambda x: x[0])

print(f"\n✅ 총 npy 파일 수 (샤딩 대상 곡 수): {len(npy_paths):,}")
if skipped_count > 0:
    print(f"⚠️ 건너뛴 파일 수: {skipped_count}")

if len(npy_paths) == 0:
    raise RuntimeError("샤딩할 npy 파일을 찾지 못했음. MEL_ROOT 경로를 확인해라.")

# ============================================================
# 샘플 파일들의 shape 및 T 분포 확인
# ============================================================
print("\n📊 샘플 파일 T(시간축 길이) 분포 확인 (처음 200곡)...")

sample_size = min(200, len(npy_paths))
t_values = []
n_mels_set = set()

for i in range(sample_size):
    sid, path = npy_paths[i]
    try:
        mel = np.load(path)
        if mel.ndim == 2:
            n_mels, T = mel.shape
            n_mels_set.add(n_mels)
            t_values.append(T)
    except Exception as e:
        print(f"[ERROR] 로드 실패: {path}, {e}")

if t_values:
    t_arr = np.array(t_values)
    print(f"\n  샘플 수: {len(t_values)}")
    print(f"  n_mels 값들: {sorted(n_mels_set)}")
    print(f"  T_min: {t_arr.min()}")
    print(f"  T_max: {t_arr.max()}")
    print(f"  T_mean: {t_arr.mean():.1f}")
    print(f"  T_median: {np.median(t_arr):.1f}")
    print(f"\n  FIXED_T={FIXED_T} 기준:")
    shorter = (t_arr < FIXED_T).sum()
    longer = (t_arr >= FIXED_T).sum()
    print(f"    - T < {FIXED_T}: {shorter}개 ({shorter/len(t_arr)*100:.1f}%) → 패딩 필요")
    print(f"    - T >= {FIXED_T}: {longer}개 ({longer/len(t_arr)*100:.1f}%) → 크롭 또는 그대로")
else:
    print("⚠️ 샘플 로드 실패")

print(f"\n📋 예상 샤드 수: {(len(npy_paths) + SHARD_SIZE - 1) // SHARD_SIZE}개")

🔍 npy 파일 스캔 중...

✅ 총 npy 파일 수 (샤딩 대상 곡 수): 50,000

📊 샘플 파일 T(시간축 길이) 분포 확인 (처음 200곡)...

  샘플 수: 200
  n_mels 값들: [48]
  T_min: 1876
  T_max: 1876
  T_mean: 1876.0
  T_median: 1876.0

  FIXED_T=1536 기준:
    - T < 1536: 0개 (0.0%) → 패딩 필요
    - T >= 1536: 200개 (100.0%) → 크롭 또는 그대로

📋 예상 샤드 수: 100개


In [3]:
# [Cell 3] 유틸 함수 정의 (crop_or_pad, 로더, 샤드 저장)

def crop_or_pad(mel: np.ndarray, fixed_t: int = FIXED_T) -> np.ndarray:
    """
    멜 스펙트로그램을 고정 길이로 crop 또는 pad 합니다.
    
    Args:
        mel: (n_mels, T_raw) 형태의 멜 스펙트로그램
        fixed_t: 출력 길이 T
    
    Returns:
        (n_mels, fixed_t) 형태의 멜 스펙트로그램
    """
    if mel.ndim != 2:
        raise ValueError(f"mel shape가 2차원이 아님: {mel.shape}")
    
    n_mels, T = mel.shape
    
    if T == fixed_t:
        return mel
    
    if T > fixed_t:
        # 길면 앞에서 fixed_t 만큼 자름 (필요시 랜덤 크롭으로 변경 가능)
        start = 0
        return mel[:, start:start + fixed_t]
    
    # T < fixed_t → 뒤쪽 0 패딩
    pad_width = fixed_t - T
    pad = np.zeros((n_mels, pad_width), dtype=mel.dtype)
    return np.concatenate([mel, pad], axis=1)


def load_mel(path: Path) -> np.ndarray:
    """
    단일 npy 파일에서 멜 스펙트로그램을 로드합니다.
    
    Returns:
        (n_mels, T) 형태의 멜 스펙트로그램
    """
    mel = np.load(path)
    
    if mel.ndim != 2:
        raise ValueError(f"mel ndim != 2: {path}, shape={mel.shape}")
    
    # (T, n_mels) 형태인 경우 Transpose
    if mel.shape[0] != ORIGINAL_N_MELS and mel.shape[1] == ORIGINAL_N_MELS:
        mel = mel.T
    
    return mel


def save_shard(shard_idx: int, song_ids: list, mels: list, shard_root: Path = SHARD_ROOT):
    """
    샤드 하나를 npz로 저장합니다.
    
    Args:
        shard_idx: 샤드 인덱스
        song_ids: list[int] - 곡 ID 리스트
        mels: list[np.ndarray] - 각각 (n_mels, FIXED_T) 형태
        shard_root: 저장 디렉토리
    
    Returns:
        저장된 파일 경로 또는 None
    """
    if len(song_ids) == 0:
        return None
    
    mels_arr = np.stack(mels, axis=0)  # (N, n_mels, FIXED_T)
    song_ids_arr = np.array(song_ids, dtype=np.int64)
    
    out_path = shard_root / f"mel_shard_{shard_idx:03d}.npz"
    np.savez_compressed(out_path, song_ids=song_ids_arr, mel=mels_arr)
    
    print(f"[SAVE] {out_path.name}  shape={mels_arr.shape}")
    return out_path


# 테스트
print("✅ 유틸 함수 정의 완료")
print(f"   - crop_or_pad: FIXED_T={FIXED_T}")
print(f"   - load_mel: ORIGINAL_N_MELS={ORIGINAL_N_MELS}")
print(f"   - save_shard: SHARD_ROOT={SHARD_ROOT}")

✅ 유틸 함수 정의 완료
   - crop_or_pad: FIXED_T=1536
   - load_mel: ORIGINAL_N_MELS=48
   - save_shard: SHARD_ROOT=C:\Users\ASUS\music_recommend\work\mel_shards


In [4]:
# [Cell 4] 샤딩 메인 루프 (tqdm으로 진행률/ETA 표시)

print(f"🚀 샤딩 시작: {len(npy_paths):,}곡 → SHARD_SIZE={SHARD_SIZE}")
print(f"   예상 샤드 수: {(len(npy_paths) + SHARD_SIZE - 1) // SHARD_SIZE}개")
print("=" * 60)

shard_meta = []       # 전체 인덱스 메타데이터
shard_song_ids = []   # 현재 샤드의 song_id 리스트
shard_mels = []       # 현재 샤드의 mel 리스트
shard_idx = 0         # 현재 샤드 인덱스

error_count = 0
success_count = 0

pbar = tqdm(npy_paths, desc="Sharding mel npy", unit="file")

for inner_global_idx, (song_id, path) in enumerate(pbar):
    try:
        mel = load_mel(path)
    except Exception as e:
        print(f"\n[ERROR] np.load 실패: song_id={song_id}, path={path}, err={e}")
        error_count += 1
        continue
    
    n_mels, T_raw = mel.shape
    
    try:
        mel_fixed = crop_or_pad(mel, FIXED_T)
    except Exception as e:
        print(f"\n[ERROR] crop_or_pad 실패: song_id={song_id}, shape={mel.shape}, err={e}")
        error_count += 1
        continue
    
    inner_idx = len(shard_song_ids)
    shard_song_ids.append(song_id)
    shard_mels.append(mel_fixed)
    
    # 메타데이터 기록
    shard_meta.append({
        "song_id": int(song_id),
        "shard_idx": int(shard_idx),
        "inner_idx": int(inner_idx),
        "t_raw": int(T_raw),
    })
    
    success_count += 1
    
    # 샤드 하나 채워지면 저장
    if len(shard_song_ids) >= SHARD_SIZE:
        save_shard(shard_idx, shard_song_ids, shard_mels)
        shard_idx += 1
        shard_song_ids = []
        shard_mels = []

# 마지막 남은 곡들 저장
if len(shard_song_ids) > 0:
    save_shard(shard_idx, shard_song_ids, shard_mels)

print("\n" + "=" * 60)
print(f"✅ 샤딩 완료!")
print(f"   - 성공: {success_count:,}곡")
print(f"   - 에러: {error_count:,}곡")
print(f"   - 생성된 샤드 수: {shard_idx + 1}개")

🚀 샤딩 시작: 50,000곡 → SHARD_SIZE=500
   예상 샤드 수: 100개


Sharding mel npy:   0%|          | 0/50000 [00:00<?, ?file/s]

[SAVE] mel_shard_000.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_001.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_002.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_003.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_004.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_005.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_006.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_007.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_008.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_009.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_010.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_011.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_012.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_013.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_014.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_015.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_016.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_017.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_018.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_019.npz  shape=(500, 48, 1536)
[SAVE] mel_shard_020.npz  shape=(500, 48

In [5]:
# [Cell 4-b] 인덱스 파일 저장 (CSV / JSON)

df_index = pd.DataFrame(shard_meta)

index_csv_path = SHARD_ROOT / "mel_shards_index.csv"
index_json_path = SHARD_ROOT / "mel_shards_index.json"

# CSV 저장
df_index.to_csv(index_csv_path, index=False)
print(f"[SAVE] index CSV: {index_csv_path}")

# JSON 저장
with open(index_json_path, "w", encoding="utf-8") as f:
    json.dump(shard_meta, f, ensure_ascii=False, indent=2)
print(f"[SAVE] index JSON: {index_json_path}")

print(f"\n📊 인덱스 통계:")
print(f"   - 총 곡 수: {len(df_index):,}")
print(f"   - 샤드 수: {df_index['shard_idx'].nunique()}개")
print(f"   - t_raw 범위: {df_index['t_raw'].min()} ~ {df_index['t_raw'].max()}")
print(f"   - t_raw 평균: {df_index['t_raw'].mean():.1f}")

print("\n📋 인덱스 샘플:")
display(df_index.head(10))

[SAVE] index CSV: C:\Users\ASUS\music_recommend\work\mel_shards\mel_shards_index.csv
[SAVE] index JSON: C:\Users\ASUS\music_recommend\work\mel_shards\mel_shards_index.json

📊 인덱스 통계:
   - 총 곡 수: 50,000
   - 샤드 수: 100개
   - t_raw 범위: 1358 ~ 1876
   - t_raw 평균: 1875.7

📋 인덱스 샘플:


,song_id,shard_idx,inner_idx,t_raw
0,3,0,0,1876
1,9,0,1,1876
2,40,0,2,1876
3,54,0,3,1876
4,57,0,4,1876
5,59,0,5,1876
6,64,0,6,1876
7,86,0,7,1876
8,96,0,8,1876
9,111,0,9,1876


In [6]:
# [Cell 5] 결과 검증 (샤드 하나 로드해서 shape/인덱스 확인)

print("🔍 결과 검증...")
print("=" * 60)

# 1. 샘플 샤드 로드
shard_files = sorted(SHARD_ROOT.glob("mel_shard_*.npz"))
print(f"\n📁 생성된 샤드 파일 수: {len(shard_files)}개")

if len(shard_files) == 0:
    raise RuntimeError("샤드 파일이 없습니다!")

sample_shard_path = shard_files[0]
data = np.load(sample_shard_path)
song_ids = data["song_ids"]
mel = data["mel"]

print(f"\n📦 샘플 샤드: {sample_shard_path.name}")
print(f"   - song_ids shape: {song_ids.shape}")
print(f"   - mel shape: {mel.shape}  (N, n_mels, FIXED_T)")
print(f"   - mel dtype: {mel.dtype}")
print(f"   - 파일 크기: {sample_shard_path.stat().st_size / 1024 / 1024:.2f} MB")

# 2. 인덱스 로드 및 확인
print("\n📋 인덱스 검증:")
df_index = pd.read_csv(SHARD_ROOT / "mel_shards_index.csv")
print(f"   - 인덱스 행 수: {len(df_index):,}")
display(df_index.head())

# 3. 인덱스와 샤드 내용 일관성 체크
print("\n🔗 인덱스-샤드 일관성 체크:")
first_row = df_index.iloc[0]
sid = first_row["song_id"]
sidx = first_row["shard_idx"]
iidx = first_row["inner_idx"]

shard_path = SHARD_ROOT / f"mel_shard_{int(sidx):03d}.npz"
d = np.load(shard_path)

if d["song_ids"][int(iidx)] == sid:
    print(f"   ✅ 검증 OK: song_id={sid} 가 shard {sidx}의 inner_idx {iidx}에 위치")
else:
    print(f"   ❌ 검증 FAIL: 불일치 발생!")

# 4. 마지막 샤드도 체크
last_row = df_index.iloc[-1]
sid_last = last_row["song_id"]
sidx_last = last_row["shard_idx"]
iidx_last = last_row["inner_idx"]

shard_path_last = SHARD_ROOT / f"mel_shard_{int(sidx_last):03d}.npz"
d_last = np.load(shard_path_last)

if d_last["song_ids"][int(iidx_last)] == sid_last:
    print(f"   ✅ 검증 OK (마지막): song_id={sid_last} 가 shard {sidx_last}의 inner_idx {iidx_last}에 위치")
else:
    print(f"   ❌ 검증 FAIL (마지막): 불일치 발생!")

# 5. 총 용량 확인
total_size_mb = sum(f.stat().st_size for f in shard_files) / 1024 / 1024
print(f"\n💾 총 샤드 용량: {total_size_mb:.2f} MB ({total_size_mb/1024:.2f} GB)")

print("\n" + "=" * 60)
print("✅ 모든 검증 완료!")

🔍 결과 검증...

📁 생성된 샤드 파일 수: 100개

📦 샘플 샤드: mel_shard_000.npz
   - song_ids shape: (500,)
   - mel shape: (500, 48, 1536)  (N, n_mels, FIXED_T)
   - mel dtype: float32
   - 파일 크기: 124.85 MB

📋 인덱스 검증:
   - 인덱스 행 수: 50,000


,song_id,shard_idx,inner_idx,t_raw
0,3,0,0,1876
1,9,0,1,1876
2,40,0,2,1876
3,54,0,3,1876
4,57,0,4,1876



🔗 인덱스-샤드 일관성 체크:
   ✅ 검증 OK: song_id=3 가 shard 0의 inner_idx 0에 위치
   ✅ 검증 OK (마지막): song_id=707978 가 shard 99의 inner_idx 499에 위치

💾 총 샤드 용량: 12484.93 MB (12.19 GB)

✅ 모든 검증 완료!
